# Nettoyage des Données — Sales Dashboard

## Objectif 

- Traiter les valeurs manquantes.
- Corriger les incohérences.
- Normaliser les données.
- Préparer un fichier propre `sales_data_cleaned.csv` pour l'analyse.

## Étapes

1. Importation des données
2. Exploration du dataset
3. Identification et traitement :
   - des valeurs manquantes
   - des anomalies (prix nuls, quantités négatives, etc.)
   - des incohérences de texte (noms de région mal orthographiés)
4. Création de nouvelles colonnes utiles (revenue, année de commande)
5. Exportation du dataset nettoyé


In [14]:
# Importation des bibliothèques
import pandas as pd
import numpy as np


In [15]:
df = pd.read_csv('../data/sales_data.csv')
df.head()

,order_id,order_date,customer_id,product_id,product_name,category,quantity,price,revenue,region
0,af838341-570a-4d93-9a47-537b89778c04,2024-05-02,325.0,1020,Parfum,Beauté,2.0,416.17,NaN,Asie
1,7848c6d8-9004-4ddc-8bfa-907736ae65f3,2024-01-19,205.0,1003,Tablette,Électronique,2.0,52.52,NaN,Amérique du Nord
2,83adf678-3cf9-49ab-8e1a-c4ddceccb42d,2022-07-25,515.0,1016,Tapis de Yoga,Sports,1.0,1075.45,NaN,Asiae
3,c5f9fbb5-05df-4b2a-83cb-fe4b5e7a794f,2023-09-13,395.0,1018,Ballon de Foot,Sports,5.0,14.72,NaN,Amérique du Sud
4,5bf9572c-612c-4ee4-b7bb-f9130f965432,2024-05-13,412.0,1010,Jean,Vêtements,5.0,1436.03,NaN,Eurpoe


In [16]:
# Dimensions 
print("Nombre de lignes et colonnes :", df.shape)


Nombre de lignes et colonnes : (12000, 10)


In [17]:
# Types de données et valeurs manquantes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   order_id      12000 non-null  object 
 1   order_date    12000 non-null  object 
 2   customer_id   11870 non-null  float64
 3   product_id    12000 non-null  int64  
 4   product_name  12000 non-null  object 
 5   category      12000 non-null  object 
 6   quantity      10445 non-null  float64
 7   price         12000 non-null  float64
 8   revenue       0 non-null      float64
 9   region        11950 non-null  object 
dtypes: float64(4), int64(1), object(5)
memory usage: 937.6+ KB


In [18]:
# stats descriptives 
df.describe(include='all')

,order_id,order_date,customer_id,product_id,product_name,category,quantity,price,revenue,region
count,12000,12000,11870.000000,12000.000000,12000,12000,10445.000000,12000.000000,0.0,11950
unique,12000,1096,NaN,NaN,25,5,NaN,NaN,NaN,8
top,e8615698-f5e9-4b79-aa30-bfa427af3163,2024-12-08,NaN,NaN,Crème Hydratante,Beauté,NaN,NaN,NaN,Amérique du Sud
freq,1,23,NaN,NaN,522,2465,NaN,NaN,NaN,1548
mean,NaN,NaN,299.584583,1012.043583,NaN,NaN,2.022212,754.823593,NaN,NaN
std,NaN,NaN,172.030253,7.228488,NaN,NaN,2.002460,433.707622,NaN,NaN
min,NaN,NaN,1.000000,1000.000000,NaN,NaN,-1.000000,0.000000,NaN,NaN
25%,NaN,NaN,153.000000,1006.000000,NaN,NaN,0.000000,381.490000,NaN,NaN
50%,NaN,NaN,300.000000,1012.000000,NaN,NaN,2.000000,758.855000,NaN,NaN
75%,NaN,NaN,446.000000,1018.000000,NaN,NaN,4.000000,1126.285000,NaN,NaN


- order_date est de type object , à convertir en datetime
- Valeurs manquantes détectées dans customer_id, quantity, et region
- revenue est vide , il devra être recalculé
- Probabilité de valeurs anormales dans quantity et price 

In [19]:
# pourcentage de valeurs manquantes
missing_percentage = df.isnull().mean() * 100
print(missing_percentage)


order_id          0.000000
order_date        0.000000
customer_id       1.083333
product_id        0.000000
product_name      0.000000
category          0.000000
quantity         12.958333
price             0.000000
revenue         100.000000
region            0.416667
dtype: float64


- customer_id : Peu de valeurs manquantes ~1%. Nous allons garder les ventes mais noter les clients inconnus , nous allons les remplacer par -1.

- quantity : Beaucoup de valeurs manquantes ~13%. Nous ne pouvons pas calculer un chiffre d’affaires sans quantité ,nous allons supprimer ces lignes.

- region : Peu de valeurs manquantes ~0,4%. Important pour la répartition régionale ➔ Remplacer par 'Inconnu'.



In [20]:
# 'customer_id' manquants par -1
df['customer_id'] = df['customer_id'].fillna(-1)

# suppression des lignes où 'quantity' est manquante
df = df.dropna(subset=['quantity'])

# 'region' manquants par 'Inconnu'
df['region'] = df['region'].fillna('Inconnu')


In [21]:
# Correction des noms de région
df['region'] = df['region'].replace({
    'Eurpoe': 'Europe',
    'Asiae': 'Asie',
    'North America': 'Amérique du Nord'
})

In [22]:
# supprimer les lignes avec quantité <= 0
df = df[df['quantity'] > 0]

# supprimer les lignes avec prix <= 0
df = df[df['price'] > 0]


In [33]:
# calcul de 'revenue'
df['revenue'] = df['quantity'] * df['price']

# conversion de 'order_date' en datetime
df['order_date'] = pd.to_datetime(df['order_date'], errors='coerce')

# extraire l'année de la commande pour des filtres futurs
df['order_year'] = df['order_date'].dt.year


In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7429 entries, 0 to 11995
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   order_id          7429 non-null   object        
 1   order_date        7429 non-null   datetime64[ns]
 2   customer_id       7429 non-null   int64         
 3   product_id        7429 non-null   int64         
 4   product_name      7429 non-null   object        
 5   category          7429 non-null   object        
 6   quantity          7429 non-null   float64       
 7   price             7429 non-null   float64       
 8   revenue           7429 non-null   float64       
 9   region            7429 non-null   object        
 10  order_year        7429 non-null   int32         
 11  order_month       7429 non-null   int32         
 12  order_month_name  7429 non-null   object        
dtypes: datetime64[ns](1), float64(3), int32(2), int64(2), object(5)
memory usage: 754.

In [35]:
df.describe()

,order_date,customer_id,product_id,quantity,price,revenue,order_year,order_month
count,7429,7429.000000,7429.000000,7429.000000,7429.000000,7429.000000,7429.000000,7429.000000
mean,2023-11-01 14:55:07.793781248,295.941984,1012.114147,3.007269,764.229509,2289.986924,2023.335173,6.528604
min,2022-04-27 00:00:00,-1.000000,1000.000000,1.000000,5.050000,5.790000,2022.000000,1.000000
25%,2023-01-31 00:00:00,148.000000,1006.000000,2.000000,399.540000,862.160000,2023.000000,4.000000
50%,2023-11-06 00:00:00,295.000000,1012.000000,3.000000,766.940000,1787.080000,2023.000000,7.000000
75%,2024-08-01 00:00:00,444.000000,1018.000000,4.000000,1130.950000,3389.000000,2024.000000,10.000000
max,2025-04-26 00:00:00,600.000000,1024.000000,5.000000,1499.830000,7497.550000,2025.000000,12.000000
std,NaN,173.613862,7.215033,1.425573,428.577690,1795.049449,0.940719,3.461947


nous avons remarqué que certaines colonnes de type identifiant `customer_id`, `product_id`, `order_year` étaient au format `float`.
ces colonnes seront transformées au type `int` pour garantir la cohérence.

In [36]:
# customer_id
df['customer_id'] = df['customer_id'].astype(int)

# product_id
df['product_id'] = df['product_id'].astype(int)

# order_year
df['order_year'] = df['order_year'].astype(int)


In [37]:
df.dtypes

order_id                    object
order_date          datetime64[ns]
customer_id                  int64
product_id                   int64
product_name                object
category                    object
quantity                   float64
price                      float64
revenue                    float64
region                      object
order_year                   int64
order_month                  int32
order_month_name            object
dtype: object

In [38]:
# fichier nettoyé
df.to_csv('../data/sales_data_cleaned.csv', index=False)